In [1]:
import pandas as pd
# import numpy as np
import datetime as dt
pd.options.display.float_format = "{:,.2f}".format

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#define file names

invoice_date = input('enter invoice date: ')
date_account = 'R1' + ' ' + invoice_date
data = 'data'
condense = 'condense'
merge = 'merge'
lineitem = 'lineitem'
file_type = '.xlsx'
print(date_account)
print(date_account+file_type)

enter invoice date: 07 03 21
R1 07 03 21
R1 07 03 21.xlsx


In [4]:
#cont define file names

x = date_account + ' ' + data + file_type
y = date_account + ' ' + condense + file_type
z = date_account + ' ' + merge + file_type
w = date_account + ' ' + lineitem + file_type

print(x +'\n'+ y +'\n'+ z +'\n' + w)

R1 07 03 21 data.xlsx
R1 07 03 21 condense.xlsx
R1 07 03 21 merge.xlsx
R1 07 03 21 lineitem.xlsx


In [5]:
#imports the raw data, keeps only the columns needed for the analysis

df = pd.read_excel(x, header=None)
df2 = df.iloc[:, [2,4,5,10,11,13,16,22,45,52,75,76,78,79,80]]

In [6]:
#names headers for the columns

df2.columns = ["account", "invoice_date", "invoice_no", "invoice_total", "ship_date", "tracking_no", "ref_1", "ref_2", "charge_type", "final_charge","receiving_party", "receiving_address", "city", "state", "zip" ]

In [7]:
#inserts a column for the sales order number but with 0 in each row 
#inserts a column for the cardCode

df2.insert(6, "sales_order_no", str(0))
df2.insert(7, "card_code", '')

In [8]:
#creates a new column called "right" which pulls off the sales order no from ref field. However need to put this in the
#sales order column
#itterates through each row and makes the column "sales order no" equal to the right 6 from the ref column so we have 
#SO number in separate column. 

right = df2['ref_1'].str[-6:]
for i, row in df2.iterrows():
    df2['sales_order_no'] = right

In [9]:
#changes sales order no to numeric
for i, row in df2.iterrows():
    df2['sales_order_no'] = df2['sales_order_no'].apply(pd.to_numeric, errors = 'coerce')
    
#converts the column "invoice date" to a datetime object
df2['invoice_date'] = pd.to_datetime(df2['invoice_date'], errors = 'coerce')

#converts invoice date to a date object
df2['invoice_date'] = df2['invoice_date'].dt.date

#converts the column "ship date" to a datetime object
df2['ship_date'] = pd.to_datetime(df2['ship_date'], errors = 'coerce')

#converts ship date to a date object
df2['ship_date'] = df2['ship_date'].dt.date
  

In [10]:
#drops the second row which is a numbered row, not needed.  
df2.drop(df2.index[0])
df2.head(2)

,account,invoice_date,invoice_no,invoice_total,ship_date,tracking_no,sales_order_no,card_code,ref_1,ref_2,charge_type,final_charge,receiving_party,receiving_address,city,state,zip
0,00009A56R1,2021-07-03,0000009A56R1271,"22,933.26",2021-06-26,1Z9A56R16753703722,NaN,,NaN,NaN,Worldwide Expedited,27.81,EQO TESTING AND CERTIFICATION,NO 23 LUXI ROAD,KUNSHAN,NaN,"215,332.00"
1,00009A56R1,2021-07-03,0000009A56R1271,"22,933.26",2021-06-26,1Z9A56R16753703722,NaN,,NaN,NaN,Peak Surcharge - Commercial,0.11,EQO TESTING AND CERTIFICATION,NO 23 LUXI ROAD,KUNSHAN,NaN,"215,332.00"


In [11]:
#check data types
df2.dtypes

account               object
invoice_date          object
invoice_no            object
invoice_total        float64
ship_date             object
tracking_no           object
sales_order_no       float64
card_code             object
ref_1                 object
ref_2                 object
charge_type           object
final_charge         float64
receiving_party       object
receiving_address     object
city                  object
state                 object
zip                  float64
dtype: object

In [12]:
#saves new dataframe excel doc. 
df2.to_excel(y, index = False)

MERGE DATA FRAMES (UPS RAW DATA WITH ORDR AND LINE ITEM TABLES)

In [13]:
#pull in ordr sheet
df_ordr = pd.read_excel('ORDR for UPS analysis US.xlsx', sheet_name = 'ORDR_only')
# df_ordr.head(2)

In [14]:
#pull in ordr line items sheet 
df_lineitems = pd.read_excel('ORDR for UPS analysis US.xlsx', sheet_name = 'ORDR_line_items')
# df_lineitems.head(2)

In [15]:
#merge ORDR and UPS data dataframes
df_merge = pd.merge(df2, df_ordr, on = ['sales_order_no'], how = 'left')

In [16]:
# df_merge.head(2)

In [17]:
#push the new DF to excel file

df_merge.to_excel(z, index = False)

UPS DATA  ANALYSIS

In [18]:
#drop the rows where final charge = 0

df_merge = df_merge[df_merge['final_charge'] != 0]

In [19]:
# print the shape of df_merge and count unique tracking numbers (after 0 charges are removed, so all tracking 
# nos with charges)
 
tracking_unique_total = df_merge.tracking_no.value_counts() 
print('dataframe shape: ' , df_merge.shape)
print('unique tracking nos: ', tracking_unique_total.count())

dataframe shape:  (3827, 22)
unique tracking nos:  1137


In [20]:
#find the new mean without the null rows. 
#aka take sum final charge divide by # of unique tracking ==> cost per carton

print('average charge overall: ', format(df_merge.final_charge.mean(), '.2f'))
print('total charges: ', format(df_merge.final_charge.sum(), ',.2f'))
print('total packages: ', format(tracking_unique_total.count(), ','))
print('average cost per package: ', format(df_merge.final_charge.sum()/tracking_unique_total.count(), '.2f'))


average charge overall:  5.99
total charges:  22,933.26
total packages:  1,137
average cost per package:  20.17


UPS INVOICE SUMMARY

In [21]:
#sum final charge by invoice number and date
print('total charges: ', format(df_merge.final_charge.sum(), ',.2f'))
print('total packages: ', format(tracking_unique_total.count(), ','))
print('average cost per package: ', format(df_merge.final_charge.sum()/tracking_unique_total.count(), '.2f'))


total charges:  22,933.26
total packages:  1,137
average cost per package:  20.17


In [22]:
#dataframe groupby object
code_grp = df_merge.groupby(['CardCode'])

#create dataframe of the code group = DIORET
df_dioret = code_grp.get_group('DIORET')

In [31]:
sum_finalcharge = df_merge[['final_charge']].sum()
invoice_by_date = df_merge.groupby(['invoice_date', 'invoice_no'])[['final_charge']].sum()
dioret_total_sum = df_dioret['final_charge'].sum()

print('invoice data:')
print(invoice_by_date)
print(sum_finalcharge)
print("online sales shipping charges total: " , format(dioret_total_sum, ',.2f'))


invoice data:
                              final_charge
invoice_date invoice_no                   
2021-07-03   0000009A56R1271     22,933.26
final_charge   22,933.26
dtype: float64
online sales shipping charges total:  19,752.65


In [24]:
# df_merge['CardCode'].value_counts()

DIONO ONLINE SALES FREIGHT SUMMARY

In [25]:
#analysis on the DIORET dataframe

dioret_total_tracking = df_dioret['tracking_no'].value_counts().count()
dioret_total_sum = df_dioret['final_charge'].sum()
print('total DIORET tracking no:', format(dioret_total_tracking, ','))
print('total DIORET sum final charge:', format(dioret_total_sum, ',.2f'))
print('average cost DIORET shipment: ', format(dioret_total_sum/dioret_total_tracking, '.2f'))

total DIORET tracking no: 969
total DIORET sum final charge: 19,752.65
average cost DIORET shipment:  20.38


In [26]:
#sums final charge by customer
print('SUM OF CHARGES BY CUSTOMER')
print(df_merge.groupby(['CardCode','CardName'])[['final_charge']].sum())

SUM OF CHARGES BY CUSTOMER
                                                    final_charge
CardCode     CardName                                           
AMAZON       Amazon                                        45.90
BRIBAB       Bringing Up Baby                               8.94
CAMB01       Cambria Recall 2020 01                       128.09
COOSER-DS    Coordinated Claim Services drop ship          16.89
DIORET       B2C Credit Card                           19,752.65
FACBOK       Facebook                                      14.96
KIDTEC       Kidmoto Technologies LLC                     524.08
MAREBAY      Ebay Market                                   36.10
MARWAL       Walmart Market Place                         892.11
MEGBAB-DS    Mega Babies Drop Ship                        151.23
MERDIS       Mercury Distributing Dba Child Source        490.25
MODNAT       Modern Natural Baby     (BRIXY)               39.22
PREALT       Prevention Alternative                        14.0

MERGE RDR1 LINE ITEM DETAIL WITH UPS

In [27]:
#merge line item table with UPS data. drop NA value rows

df_detail = pd.merge(df_lineitems, df2, on = ['sales_order_no'], how = 'left')
df_detail = df_detail.dropna()
# df_detail.head(2)

In [28]:
#push line item merged table to excel doc
df_detail.to_excel(w, index = False )